In [2]:
# 完成对20类新闻数据集，通过贝叶斯分析，实现分类预测，理解贝叶斯公式计算方法，得出，精确率，召回率，F1-score,并理解计算某个分类的AUC值的操作
import time

from sklearn.datasets import load_iris, fetch_20newsgroups, fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
news = fetch_20newsgroups(subset='all',data_home='data2')# 加载数据集
print(len(news.data))# 查看数据集大小
print(news.data[0])# 查看数据集中的一个样本
print(news.target[0:5])# 查看数据集中的一个样本的标签
print(np.unique(news.target))# 查看数据集的标签种类
print(news.target_names)# 查看数据集的标签种类名称


18846
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!


[10  3 17  3  4]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
['alt.atheis

In [3]:
x_train, x_test, y_train, y_test = train_test_split(news.data, news.target, test_size=0.25, random_state=1)# 划分数据集
tf = TfidfVectorizer()# 对数据集进行特征抽取
x_train = tf.fit_transform(x_train)# 训练特征抽取器
print(len(tf.get_feature_names_out()))# 查看特征抽取器抽取的特征数量

153196


In [4]:
print(tf.get_feature_names_out()[100000])# 查看特征抽取器抽取的特征名称

murky


In [5]:
print(tf.get_feature_names_out()[0:10])# 查看特征抽取器抽取的特征名称

['00' '000' '0000' '00000' '0000000004' '0000000005' '0000000667'
 '0000001200' '000003' '000005102000']


In [6]:
print(tf.get_feature_names_out()[100000:100000+10])# 查看特征抽取器抽取的特征名称

['murky' 'murmurs' 'murnane' 'murph' 'murphey' 'murphy' 'murr11' 'murray'
 'murray_craven' 'murrayfield']


In [7]:
mlt = MultinomialNB(alpha=1.0)# 创建贝叶斯分类器
start=time.time()# 开始计时
mlt.fit(x_train, y_train) # 训练模型
end=time.time()# 结束计时
end-start # 查看训练时间

0.14171338081359863

In [8]:
x_transform_test = tf.transform(x_test) # 对测试集进行特征抽取
print(len(tf.get_feature_names_out())) # 查看特征抽取器抽取的特征数量

153196


In [9]:
x_transform_test.shape# 查看测试集的特征数量

(4712, 153196)

In [10]:
start=time.time()# 开始计时
y_predict = mlt.predict(x_transform_test)# 预测测试集
print("预测的前面10篇文章类别为：", y_predict[0:10])# 查看预测结果
print("准确率为：", mlt.score(x_transform_test, y_test))# 查看准确率
end=time.time()# 结束计时
end-start #预测时间

预测的前面10篇文章类别为： [16 19 18  1  9 15  1  2 16 13]
准确率为： 0.8518675721561969


0.06778645515441895

In [11]:
len(y_predict)# 查看预测结果的数量

4712

In [12]:
print(classification_report(y_test, y_predict,
      target_names=news.target_names))# 查看精确率，召回率，F1-score

                          precision    recall  f1-score   support

             alt.atheism       0.91      0.77      0.83       199
           comp.graphics       0.83      0.79      0.81       242
 comp.os.ms-windows.misc       0.89      0.83      0.86       263
comp.sys.ibm.pc.hardware       0.80      0.83      0.81       262
   comp.sys.mac.hardware       0.90      0.88      0.89       234
          comp.windows.x       0.92      0.85      0.88       230
            misc.forsale       0.96      0.67      0.79       257
               rec.autos       0.90      0.87      0.88       265
         rec.motorcycles       0.90      0.95      0.92       251
      rec.sport.baseball       0.89      0.96      0.93       226
        rec.sport.hockey       0.95      0.98      0.96       262
               sci.crypt       0.76      0.97      0.85       257
         sci.electronics       0.84      0.80      0.82       229
                 sci.med       0.97      0.86      0.91       249
         

In [13]:
y_test.shape# 查看测试集的真实标签数量

(4712,)

In [14]:
y_test1 = np.where(y_test == 0, 1, 0)# 将测试集的真实标签转换为0和1
print(y_test1.sum())# 查看转换后的标签数量

199


In [15]:
print(y_test1[0:20])# 查看转换后的标签

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]


In [16]:
y_predict1 = np.where(y_predict == 0, 1, 0)# 将预测结果转换为0和1
print(y_predict1.sum())# 查看转换后的预测结果数量

168


In [17]:
y_predict1[0:20]# 查看转换后的预测结果

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [18]:
(y_test1*y_predict1).sum()# 查看正确预测的数量

np.int64(153)

In [19]:
max(y_test),min(y_test)# 查看测试集的真实标签的最大值和最小值

(np.int64(19), np.int64(0))

In [20]:
y_test1 = np.where(y_test == 5, 1, 0)# 将测试集的真实标签转换为0和1
print(y_test1.sum()) #label为5的样本数
y_predict1 = np.where(y_predict == 5, 1, 0)# 将预测结果转换为0和1
print(y_predict1.sum())# 查看转换后的预测结果数量
print("AUC指标：", roc_auc_score(y_test1, y_predict1))# 查看AUC值

230
214
AUC指标： 0.924078924393225


In [21]:
y_test1,y_predict1# 查看转换后的标签和预测结果

(array([0, 0, 0, ..., 0, 0, 0], shape=(4712,)),
 array([0, 0, 0, ..., 0, 0, 0], shape=(4712,)))

In [22]:
FP=np.where((np.array(y_test1)-np.array(y_predict1))==-1,1,0).sum() #FP是196
TP=y_predict1.sum()-FP #TP是196
print(TP)
FN=np.where((np.array(y_test1)-np.array(y_predict1))==1,1,0).sum() #FN是34
print(FN)#FN是1
TN=np.where(y_test1==0,1,0).sum()-FP  #4464
print(TN)

196
34
4464


In [23]:
TP/(TP+FP)#精确率

np.float64(0.9158878504672897)

In [24]:
TP/(TP+FN)#召回率

np.float64(0.8521739130434782)

In [25]:
2*TP/(2*TP+FP+FN)#F1-score

np.float64(0.8828828828828829)